## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy
duet1 = sy.duet("ff666f18ef6e671b4e03d545833f2134")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 1912f0cdf422d82a117b09ba86f59080

♫♫♫ > ...waiting for partner to connect...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [2]:
duet2 = sy.duet("bd2681f95556591e6687b3935c7a0412")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: f8ac1de9f28c439b56b7ba642f91d424

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [3]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 322b7b11d20b44faa2a0c61e97a921d2>,[DO1 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: f1f767cee96e45d88c16552acf7cf02a>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>


In [4]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 0702fd696c874562a96199fcaf09a96b>,[DO2 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: a5244cc8ac964d08a03b8ea7433f6374>,[DO2 training y data],number of samples and features,<class 'torch.Tensor'>


## Modelling

In [5]:
data1_ptr = duet1.store[0]
target1_ptr = duet1.store[1]
data2_ptr = duet2.store[0]
target2_ptr = duet2.store[1]

print(data1_ptr)
print(target1_ptr)
print(data2_ptr)
print(target2_ptr)

### Create Base Model

In [47]:
import torch
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score, mean_poisson_deviance
from scipy import stats

In [7]:
#in_dim = 39
#out_dim = 1

In [8]:
#class SyNet(sy.Module):
    #def __init__(self, torch_ref):
        #super(SyNet, self).__init__(torch_ref=torch_ref)
        #self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    #def forward(self, x):
        #x = self.linear(x)
        #return x

In [9]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.layer_1 = self.torch_ref.nn.Linear(39, 31)
        self.layer_out = self.torch_ref.nn.Linear(31, 1)
        # Define proportion or neurons to dropout
        self.dropout_1 = self.torch_ref.nn.Dropout(0.12409392594394411)
        self.relu = self.torch_ref.nn.ReLU()
    
    def forward(self, inputs):
        x = self.dropout_1(self.relu(self.layer_1(inputs)))
        x = self.layer_out(x)

        return (x)

    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.layer_out(x)

        return (x)

### Training

Try training loop from MNIST example

In [67]:
def train_MNIST(model, torch_ref, train_loader, optimizer, epoch, args, train_data_length):
    # + 0.5 lets us math.ceil without the import
    train_batches = round((train_data_length / args["batch_size"]) + 0.5)
    print(f"> Running train in {train_batches} batches")
    if model.is_local:
        print("Training requires remote model")
        return

    model.train()

    for batch_idx, data in enumerate(train_loader):
        data_ptr, target_ptr = data[0], data[1]
        optimizer.zero_grad()
        output = model(data_ptr)
        loss = torch_ref.nn.functional.nll_loss(output, target_ptr)
        loss.backward()
        optimizer.step()
        loss_item = loss.item()
        train_loss = duet.python.Float(0)  # create a remote Float we can use for summation
        train_loss += loss_item
        if batch_idx % args["log_interval"] == 0:
            local_loss = None
            local_loss = loss_item.get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5
            )
            if local_loss is not None:
                print("Train Epoch: {} {} {:.4}".format(epoch, batch_idx, local_loss))
            else:
                print("Train Epoch: {} {} ?".format(epoch, batch_idx))
        if batch_idx >= train_batches - 1:
            print("batch_idx >= train_batches, breaking")
            break
        if args["dry_run"]:
            break

In [76]:
# The DO has kindly let us initialise a DataLoader for their training set
train_kwargs = {
    "batch_size": args["batch_size"],
}
#train_data_ptr = duet1.store[0]
train_loader_ptr = remote_torch1.utils.data.DataLoader(data1_ptr,**train_kwargs)

In [78]:
# normally we would not necessarily know the length of a remote dataset so lets ask for it
# so we can pass that to our training loop and know when to stop
def get_train_length(train_data_ptr):
    train_data_length = len(train_data_ptr)
    return train_data_length

try:
    if train_data_length is None:
        train_data_length = get_train_length(data1_ptr)
except NameError:
        train_data_length = get_train_length(data1_ptr)

print(f"Training Dataset size is: {train_data_length}")

In [10]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    #criterion = torch_ref.nn.functional.poisson_nll_loss(log_input= True, full= True)

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        #loss = torch_ref.nn.functional.nll_loss(output, target_ptr)
        loss = torch_ref.nn.functional.poisson_nll_loss(output, target_ptr, log_input= True, full= True)   
        #loss = torch_ref.nn.functional.mse_loss(output, target_ptr)
        #loss = criterion(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [11]:
import torch as th
import numpy as np

Create a local model

In [12]:
base_model = SyNet(torch)

Send the model to DO1

In [13]:
remote_model1 = base_model.send(duet1)

Create alias for D01's instance of torch called `remote_torch1` so we can refer to the local torch as `torch` and any operation we want to do remotely as `remote_torch1`. Remember, the return values from `remote_torch1` are `Pointers`, not the real objects. They mostly act the same when using them with other `Pointers` but you can't mix them with local torch objects.

In [55]:
remote_torch1 = duet1.torch

Set arguments for training

In [66]:
# Lets define a few settings which are from the original MNIST example command-line args
args = {
    "batch_size": 64,
    "test_batch_size": 1000,
    "epochs": 14,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": False,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": True,
}

CUDA check cells below doesn't appear to works

In [62]:
# lets ask to see if our Data Owner has CUDA
#has_cuda = False
#has_cuda_ptr = remote_torch1.cuda.is_available()
#has_cuda = bool(has_cuda_ptr.get())
#print(has_cuda)

In [63]:
#use_cuda = not args["no_cuda"] and has_cuda
# now we can set the seed
#remote_torch.manual_seed(args["seed"])

#device = remote_torch.device("cuda" if use_cuda else "cpu")
#print(f"Data Owner device is {device.type.get()}")

In [64]:
# if we have CUDA lets send our model to the GPU
#if has_cuda:
    #model.cuda(device)
#else:
    #model.cpu()

In [65]:
params = remote_model1.parameters()

In [56]:
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

In [15]:
iteration = 25
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 0.9675072431564331
Epoch 1 loss 0.25259271264076233
Epoch 2 loss 0.2833402454853058
Epoch 3 loss 0.3596494495868683
Epoch 4 loss 0.36820387840270996
Epoch 5 loss 0.33501142263412476
Epoch 6 loss 0.28644120693206787
Epoch 7 loss 0.2435450702905655
Epoch 8 loss 0.2221793532371521
Epoch 9 loss 0.22975605726242065
Epoch 10 loss 0.24947024881839752
Epoch 11 loss 0.24752941727638245
Epoch 12 loss 0.23148126900196075
Epoch 13 loss 0.2219582498073578
Epoch 14 loss 0.2205892652273178
Epoch 15 loss 0.22509416937828064
Epoch 16 loss 0.22930310666561127
Epoch 17 loss 0.23033839464187622
Epoch 18 loss 0.22836017608642578
Epoch 19 loss 0.22428017854690552
Epoch 20 loss 0.2204708307981491
Epoch 21 loss 0.21759475767612457
Epoch 22 loss 0.21869152784347534
Epoch 23 loss 0.22089259326457977
Epoch 24 loss 0.2223421037197113


Train on Data Owner 2 data

In [16]:
remote_model2 = base_model.send(duet2)

In [17]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

In [18]:
iteration = 25
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 0.967623233795166
Epoch 1 loss 0.25391456484794617
Epoch 2 loss 0.28629398345947266
Epoch 3 loss 0.36338305473327637
Epoch 4 loss 0.37011367082595825
Epoch 5 loss 0.33514249324798584
Epoch 6 loss 0.28645068407058716
Epoch 7 loss 0.24405555427074432
Epoch 8 loss 0.22424325346946716
Epoch 9 loss 0.23271997272968292
Epoch 10 loss 0.25173133611679077
Epoch 11 loss 0.24873320758342743
Epoch 12 loss 0.23339159786701202
Epoch 13 loss 0.22353337705135345
Epoch 14 loss 0.2234545797109604
Epoch 15 loss 0.226954847574234
Epoch 16 loss 0.23202267289161682
Epoch 17 loss 0.23258498311042786
Epoch 18 loss 0.23067247867584229
Epoch 19 loss 0.22616545855998993
Epoch 20 loss 0.22213506698608398
Epoch 21 loss 0.21967430412769318
Epoch 22 loss 0.22062432765960693
Epoch 23 loss 0.2231285572052002
Epoch 24 loss 0.2244308739900589


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [19]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[-0.1263,  0.0106, -0.1283,  ..., -0.0751, -0.0363,  0.0688],
        [-0.0893, -0.1118,  0.0577,  ..., -0.0227,  0.0760, -0.0616],
        [-0.1561,  0.0440,  0.0982,  ..., -0.1406, -0.1197,  0.0893],
        ...,
        [-0.0721, -0.1279,  0.0421,  ..., -0.0668,  0.0589, -0.0801],
        [ 0.1059, -0.1079,  0.0716,  ..., -0.0889,  0.0449,  0.1104],
        [-0.1188, -0.0803, -0.0919,  ...,  0.0567,  0.0474, -0.0054]],
       requires_grad=True), Parameter containing:
tensor([-0.1167,  0.1575, -0.0918,  0.0347,  0.0797, -0.0727,  0.1446, -0.0194,
        -0.0313, -0.1159, -0.1392,  0.0880,  0.0081,  0.0422,  0.1128,  0.1071,
        -0.0674, -0.1275,  0.0218, -0.0305,  0.1316, -0.1534,  0.0911, -0.1578,
         0.1431,  0.0996, -0.0701,  0.0257, -0.0502,  0.1555,  0.1305],
       requires_grad=True), Parameter containing:
tensor([[ 0.0827,  0.1264,  0.0488,  0.1081, -0.0350, -0.0663,  0.1164,  0.1190,
          0.0412, -0.0337, 

As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [20]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('layer_1.weight', tensor([[-0.6818, -0.5457, -0.6829,  ..., -0.0751, -0.0363, -0.4513],
        [-0.6459, -0.6684, -0.4989,  ..., -0.5793, -0.4806, -0.6182],
        [-0.7127, -0.5126, -0.4585,  ..., -0.6972, -0.6763, -0.4672],
        ...,
        [-0.1361,  0.0051,  0.2463,  ...,  0.2685,  0.4015,  0.1573],
        [ 0.1760,  0.1334,  0.3674,  ...,  0.4384,  0.4327,  0.3927],
        [-0.6754, -0.6369, -0.6486,  ..., -0.4999, -0.5092, -0.5617]])), ('layer_1.bias', tensor([-0.6731, -0.3991, -0.6485, -0.5219, -0.2134, -0.1244, -0.4120, -0.5761,
        -0.5867, -0.4162, -0.6958, -0.4686,  0.2587, -0.5144, -0.4438, -0.4495,
         0.2097, -0.4207, -0.5348, -0.2746, -0.4262, -0.5177, -0.2549, -0.7144,
        -0.4135,  0.4458, -0.6268,  0.3160,  0.1464,  0.4895, -0.4261])), ('layer_out.weight', tensor([[-0.4733, -0.4303, -0.5078, -0.4486,  0.5210, -0.0042, -0.4402, -0.4376,
         -0.5091,  0.5471, -0.4599, -0.4442, -0.1059, -0.3847, -0.4437, -0.5383,
         -0.1769, 

In [21]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('layer_1.weight', tensor([[-0.6815, -0.5456, -0.6830,  ..., -0.0751, -0.0363,  0.0688],
        [-0.6459, -0.6684, -0.4989,  ..., -0.5793, -0.4806, -0.6182],
        [-0.7127, -0.5126, -0.4585,  ..., -0.6972, -0.6763, -0.4672],
        ...,
        [-0.1358,  0.0089,  0.2510,  ...,  0.2675,  0.3831,  0.1096],
        [ 0.1746,  0.1396,  0.3737,  ...,  0.4374,  0.3918,  0.3371],
        [-0.6754, -0.6369, -0.6486,  ..., -0.4999, -0.5092, -0.5617]])), ('layer_1.bias', tensor([-0.6731, -0.3991, -0.6485, -0.5219, -0.2148, -0.1291, -0.4120, -0.5761,
        -0.5859, -0.4127, -0.6958, -0.4686,  0.2577, -0.5144, -0.4438, -0.4495,
         0.2062, -0.4198, -0.5348, -0.2766, -0.4262, -0.5168, -0.2557, -0.7144,
        -0.4135,  0.4448, -0.6268,  0.3132,  0.1449,  0.4865, -0.4261])), ('layer_out.weight', tensor([[-4.7322e-01, -4.3025e-01, -5.0781e-01, -4.4856e-01,  5.1728e-01,
          3.5433e-04, -4.4024e-01, -4.3761e-01, -5.0658e-01,  5.4685e-01,
         -4.5985e-01, -4.4421e-0

In [22]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [23]:
avg_updates = OrderedDict()
avg_updates["layer_1.weight"] = (
    remote_model1_updates["layer_1.weight"] + remote_model2_updates["layer_1.weight"]
) / 2
avg_updates["layer_1.bias"] = (
    remote_model1_updates["layer_1.bias"] + remote_model2_updates["layer_1.bias"]
) / 2
avg_updates["layer_out.weight"] = (
    remote_model1_updates["layer_out.weight"] + remote_model2_updates["layer_out.weight"]
) / 2
avg_updates["layer_out.bias"] = (
    remote_model1_updates["layer_out.bias"] + remote_model2_updates["layer_out.bias"]
) / 2

print(avg_updates)

OrderedDict([('layer_1.weight', tensor([[-0.6817, -0.5456, -0.6830,  ..., -0.0751, -0.0363, -0.1913],
        [-0.6459, -0.6684, -0.4989,  ..., -0.5793, -0.4806, -0.6182],
        [-0.7127, -0.5126, -0.4585,  ..., -0.6972, -0.6763, -0.4672],
        ...,
        [-0.1360,  0.0070,  0.2486,  ...,  0.2680,  0.3923,  0.1334],
        [ 0.1753,  0.1365,  0.3706,  ...,  0.4379,  0.4122,  0.3649],
        [-0.6754, -0.6369, -0.6486,  ..., -0.4999, -0.5092, -0.5617]])), ('layer_1.bias', tensor([-0.6731, -0.3991, -0.6485, -0.5219, -0.2141, -0.1268, -0.4120, -0.5761,
        -0.5863, -0.4144, -0.6958, -0.4686,  0.2582, -0.5144, -0.4438, -0.4495,
         0.2079, -0.4203, -0.5348, -0.2756, -0.4262, -0.5173, -0.2553, -0.7144,
        -0.4135,  0.4453, -0.6268,  0.3146,  0.1456,  0.4880, -0.4261])), ('layer_out.weight', tensor([[-0.4733, -0.4303, -0.5078, -0.4486,  0.5191, -0.0019, -0.4402, -0.4376,
         -0.5079,  0.5470, -0.4599, -0.4442, -0.1074, -0.3847, -0.4437, -0.5383,
         -0.1758, 

### Load aggregated weights

In [24]:
combined_model = SyNet(torch)

In [25]:
combined_model.load_state_dict(avg_updates)

In [26]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [27]:
import pandas as pd

In [28]:
df_X_val_0 = pd.read_csv('ETL/X_val_0.csv')
df_y_val_0 = pd.read_csv('ETL/y_val_0.csv')
X_val_0_np = df_X_val_0.to_numpy()
y_val_0_np = df_y_val_0.to_numpy()
X_val_0 = th.from_numpy(X_val_0_np).float()
y_val_0 = th.from_numpy(y_val_0_np).float()

In [49]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = combined_model(sample)

        preds.append(y_hat)
preds = [a.squeeze().tolist() for a in preds]
preds_exp=np.exp(preds)

In [50]:
def test_statistics(y_test, y_pred_list_exp):
    mpd = mean_poisson_deviance(y_test, y_pred_list_exp)
    mse = mean_squared_error(y_test, y_pred_list_exp)
    r_square = r2_score(y_test, y_pred_list_exp)
    print("Mean Poisson Deviance :",mpd)
    print("Mean Squared Error :",mse)
    print("R^2 :",r_square)
    print(stats.describe(y_pred_list_exp))

test_statistics(y_val_0, preds_exp)    

Mean Poisson Deviance : 0.34470368907283583
Mean Squared Error : 0.059390698372633506
R^2 : -0.04993585845490034
DescribeResult(nobs=27121, minmax=(0.020573574537936536, 0.49227459518370614), mean=0.08662207216265036, variance=0.0016356821994002462, skewness=1.6160389658242844, kurtosis=4.368956680530511)


In [51]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = base_model(sample)

        preds.append(y_hat)
preds = [a.squeeze().tolist() for a in preds]
preds_exp=np.exp(preds)

In [52]:
test_statistics(y_val_0, preds_exp)

Mean Poisson Deviance : 1.8362483034975676
Mean Squared Error : 0.8924787924057628
R^2 : -14.777647219738979
DescribeResult(nobs=27121, minmax=(0.7189298303256256, 1.2776400909436074), mean=0.9639654118311258, variance=0.005044648400737673, skewness=0.2569394470817801, kurtosis=-0.12768607416552769)


## Comparison to classical linear regression on centralised data

In [ ]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [ ]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [ ]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [ ]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [ ]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)